# Домашка 5. NN

## Autoencoder
1. Значимо изменить архитектуру модели (2 балла)
    * Добавил еще слои
    * Добавил нормализацию
    * Добавил дропаут
2. Пробить скор бейзлайна DSSM модели 0.075. (2 балла)
    * Побил! Скор = 0.875 
4. Обернуть в сервис, который должен  критериям читаемости и воспоизводимости. (5 балла)
   * обернул, запустил, тут пояснял, что происходит

In [77]:
import numpy as np
import os
import pandas as pd
import math

import pickle

from scipy.sparse import csr_matrix


from random import random

from tqdm import tqdm
import warnings


from dev_eval import read_kion_dataset
from rectools.dataset import Dataset as RecDataset


from IPython.display import display, clear_output

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader


from aerecommender import AERecommender

# pd.set_option("display.max_columns", None)
warnings.filterwarnings("ignore")
tqdm.pandas()

# Data

Как всегда...

In [85]:
interactions_df = pd.read_csv("../data/data_original/interactions.csv", parse_dates=["last_watch_dt"]).rename(
    columns={"last_watch_dt": "datetime", "total_dur": "weight"}
)
users_df = pd.read_csv("../data/data_original/users.csv")
items_df = pd.read_csv("../data/data_original/items.csv")

# Препроцессинг

Удалим слишком далекие транзакции
Оставляем пользователей с больше, чем 5 просмотрами

In [4]:
interactions_df = interactions_df[interactions_df["datetime"] < "2021-04-01"]

In [5]:
users_interactions_count_df = interactions_df.groupby(["user_id", "item_id"]).size().groupby("user_id").size()
print("# users: %d" % len(users_interactions_count_df))
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[
    ["user_id"]
]
print("# users with at least 5 interactions: %d" % len(users_with_enough_interactions_df))

# users: 86614
# users with at least 5 interactions: 14563


In [6]:
print("# of interactions: %d" % len(interactions_df))
interactions_from_selected_users_df = interactions_df.merge(
    users_with_enough_interactions_df, how="right", left_on="user_id", right_on="user_id"
)
print("# of interactions from users with at least 5 interactions: %d" % len(interactions_from_selected_users_df))

# of interactions: 263874
# of interactions from users with at least 5 interactions: 142670


Сглаживаем признак "время просмотра"

In [7]:
def smooth_user_preference(x):
    return math.log(1 + x, 2)


interactions_full_df = (
    interactions_from_selected_users_df.groupby(["user_id", "item_id"])["watched_pct"]
    .sum()
    .apply(smooth_user_preference)
    .reset_index()
)
print("# of unique user/item interactions: %d" % len(interactions_full_df))
interactions_full_df.head(10)

# of unique user/item interactions: 142670


,user_id,item_id,watched_pct
0,21,849,6.375039
1,21,4345,6.658211
2,21,10283,6.658211
3,21,12261,6.658211
4,21,15997,6.658211
5,32,952,6.044394
6,32,4382,4.954196
7,32,4807,6.658211
8,32,10436,6.658211
9,32,12132,6.658211


# Train test split

In [8]:
interactions_train_df, interactions_test_df = train_test_split(
    interactions_full_df, stratify=interactions_full_df["user_id"], test_size=0.20, random_state=42
)

print("# interactions on Train set: %d" % len(interactions_train_df))
print("# interactions on Test set: %d" % len(interactions_test_df))

# interactions on Train set: 114136
# interactions on Test set: 28534


In [9]:
# Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_full_df.set_index("user_id")
interactions_train_indexed_df = interactions_train_df.set_index("user_id")
interactions_test_indexed_df = interactions_test_df.set_index("user_id")

In [10]:
def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[person_id]["item_id"]
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

# Обучение модели

Создадим класс для оценки модели.

In [11]:
# Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100


class ModelEvaluator:
    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interactions_full_indexed_df)
        all_items = set(articles_df["item_id"])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):
        try:
            index = next(i for i, c in enumerate(recommended_items) if c == item_id)
        except:
            index = -1
        hit = int(index in range(0, topn))
        return hit, index

    def evaluate_model_for_user(self, model, person_id):
        # Getting the items in test set
        interacted_values_testset = interactions_test_indexed_df.loc[person_id]
        if type(interacted_values_testset["item_id"]) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset["item_id"])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset["item_id"])])
        interacted_items_count_testset = len(person_interacted_items_testset)

        # Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(
            person_id, items_to_ignore=get_items_interacted(person_id, interactions_train_indexed_df), topn=10000000000
        )

        hits_at_5_count = 0
        hits_at_10_count = 0
        # For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            # Getting a random sample (100) items the user has not interacted
            # (to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(
                person_id, sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, seed=item_id % (2**32)
            )

            # Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            # Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df["item_id"].isin(items_to_filter_recs)]
            valid_recs = valid_recs_df["item_id"].values
            # Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        # Recall is the rate of the interacted items that are ranked among the Top-N recommended items,
        # when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {
            "hits@5_count": hits_at_5_count,
            "hits@10_count": hits_at_10_count,
            "interacted_count": interacted_items_count_testset,
            "recall@5": recall_at_5,
            "recall@10": recall_at_10,
        }
        return person_metrics

    def evaluate_model(self, model):
        # print('Running evaluation for users')
        people_metrics = []
        for idx, person_id in enumerate(tqdm(list(interactions_test_indexed_df.index.unique().values))):
            # if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, person_id)
            person_metrics["user_id"] = person_id
            people_metrics.append(person_metrics)
        print("%d users processed" % idx)

        detailed_results_df = pd.DataFrame(people_metrics).sort_values("interacted_count", ascending=False)

        global_recall_at_5 = detailed_results_df["hits@5_count"].sum() / float(
            detailed_results_df["interacted_count"].sum()
        )
        global_recall_at_10 = detailed_results_df["hits@10_count"].sum() / float(
            detailed_results_df["interacted_count"].sum()
        )

        global_metrics = {
            "modelName": model.get_model_name(),
            "recall@5": global_recall_at_5,
            "recall@10": global_recall_at_10,
        }
        return global_metrics, detailed_results_df


model_evaluator = ModelEvaluator()

И саму модель

__ВАЖНО__  Тут как раз улучшенная модель с дополнительными слоями, нормализацией, дропаутом.

In [12]:
# Constants
SEED = 42  # random seed for reproducibility
LR = 1e-3  # learning rate, controls the speed of the training
WEIGHT_DECAY = 0.01  # lambda for L2 reg. ()
NUM_EPOCHS = 200  # num training epochs (how many times each instance will be processed)
GAMMA = 0.9995  # learning rate scheduler parameter
BATCH_SIZE = 3000  # training batch size
EVAL_BATCH_SIZE = 3000  # evaluation batch size.
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  #'cuda' # device to make the calculations on

In [13]:
class Model(nn.Module):
    def __init__(self, in_and_out_features=8287):
        super().__init__()
        self.in_and_out_features = in_and_out_features
        self.hidden_size1 = 1000
        self.hidden_size2 = 500

        self.encoder = nn.Sequential(
            nn.Linear(in_and_out_features, self.hidden_size1),
            nn.BatchNorm1d(self.hidden_size1),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(self.hidden_size1, self.hidden_size2),
            nn.BatchNorm1d(self.hidden_size2),
            nn.ReLU(),
        )

        self.decoder = nn.Sequential(
            nn.Linear(self.hidden_size2, self.hidden_size1),
            nn.BatchNorm1d(self.hidden_size1),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(self.hidden_size1, in_and_out_features),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [14]:
torch.manual_seed(SEED)  # Fix random seed to have reproducible weights of model layers

model = Model()
model.to(DEVICE)

# Initialize GD method, which will update the weights of the model
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# Initialize learning rate scheduler, which will decrease LR according to some rule
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)


def rmse_for_sparse(x_pred, x_true):
    mask = x_true > 0
    sq_diff = (x_pred * mask - x_true) ** 2
    mse = sq_diff.sum() / mask.sum()
    return mse ** (1 / 2)

# Преобразование данных в матрицу

In [15]:
total_df = interactions_train_df.append(interactions_test_indexed_df.reset_index())
total_df["user_id"], users_keys = total_df.user_id.factorize()
total_df["item_id"], items_keys = total_df.item_id.factorize()

train_encoded = total_df.iloc[: len(interactions_train_df)].values
test_encoded = total_df.iloc[len(interactions_train_df) :].values

In [16]:
shape = [int(total_df["user_id"].max() + 1), int(total_df["item_id"].max() + 1)]
X_train = csr_matrix((train_encoded[:, 2], (train_encoded[:, 0], train_encoded[:, 1])), shape=shape).toarray()
X_test = csr_matrix((test_encoded[:, 2], (test_encoded[:, 0], test_encoded[:, 1])), shape=shape).toarray()

 ## Создание датасета и data loader'ов

In [17]:
# Initialize the DataObject, which must return an element (features vector x and target value y)
# for a given idx. This class must also have a length atribute
class UserOrientedDataset(Dataset):
    def __init__(self, X):
        super().__init__()  # to initialize the parent class
        self.X = X.astype(np.float32)
        self.len = len(X)

    def __len__(self):  # We use __func__ for implementing in-built python functions
        return self.len

    def __getitem__(self, index):
        return self.X[index]

In [18]:
# Initialize DataLoaders - objects, which sample instances from DataObject-s
train_dl = DataLoader(UserOrientedDataset(X_train), batch_size=BATCH_SIZE, shuffle=True)

test_dl = DataLoader(UserOrientedDataset(X_test), batch_size=EVAL_BATCH_SIZE, shuffle=False)

dls = {"train": train_dl, "test": test_dl}

## Обучение

In [19]:
# Training loop
metrics_dict = {
    "Epoch": [],
    "Train RMSE": [],
    "Test RMSE": [],
}

# Train loop
for epoch in range(NUM_EPOCHS):
    metrics_dict["Epoch"].append(epoch)
    for stage in ["train", "test"]:
        with torch.set_grad_enabled(stage == "train"):  # Whether to start building a graph for a backward pass
            if stage == "train":
                model.train()  # Enable some "special" layers (will speak about later)
            else:
                model.eval()  # Disable some "special" layers (will speak about later)

            loss_at_stage = 0
            for batch in dls[stage]:
                # batch = batch.to(DEVICE)
                x_pred = model(batch)  # forward pass: model(x_batch) -> calls forward()
                loss = rmse_for_sparse(x_pred, batch)  # ¡Important! y_pred is always the first arg
                if stage == "train":
                    loss.backward()  # Calculate the gradients of all the parameters wrt loss
                    optimizer.step()  # Update the parameters
                    scheduler.step()
                    optimizer.zero_grad()  # Zero the saved gradient
                loss_at_stage += loss.item() * len(batch)
            rmse_at_stage = (loss_at_stage / len(dls[stage].dataset)) ** (1 / 2)
            metrics_dict[f"{stage.title()} RMSE"].append(rmse_at_stage)

    if (epoch == NUM_EPOCHS - 1) or epoch % 10 == 9:
        clear_output(wait=True)
        display(pd.DataFrame(metrics_dict))

,Epoch,Train RMSE,Test RMSE
0,0,2.107293,2.305213
1,1,1.655865,2.304711
2,2,1.344161,2.314712
3,3,1.205986,2.226847
4,4,1.119310,2.017134
...,...,...,...
195,195,0.589207,1.247050
196,196,0.589384,1.245484
197,197,0.588815,1.268253
198,198,0.590267,1.239051


## Оценка

In [20]:
with torch.no_grad():
    X_pred = model(torch.Tensor(torch.Tensor(X_test).to(DEVICE)))
X_pred

tensor([[ 5.0434e+00,  6.3449e+00,  4.9155e+00,  ..., -3.8341e-01,
         -3.1926e-01,  4.9161e-01],
        [ 2.6571e-01,  3.5488e+00,  6.9571e-01,  ..., -1.8359e-01,
          3.0614e-02, -1.0697e-01],
        [ 1.1215e+00,  3.9133e+00,  8.8670e-01,  ..., -1.4635e-01,
         -1.4611e-01,  1.8826e-02],
        ...,
        [ 4.4887e-01,  3.6888e+00,  8.4567e-01,  ..., -2.7533e-01,
          3.2975e-03, -3.8156e-02],
        [ 1.6066e+00,  4.5655e+00,  1.0473e+00,  ...,  1.2819e-01,
         -7.0163e-02, -7.1190e-02],
        [ 1.6633e+00,  4.3546e+00,  1.1983e+00,  ..., -6.9802e-02,
         -1.0646e-01, -2.5234e-01]])

Готовим маппинг юзеров и лист популярного для модели

In [86]:
user_id2uid = {user: id for id, user in enumerate(interactions_full_indexed_df.index.unique().tolist())}

name_popular = "../models/pickle_data/popular.pickle"
loaded_popular = pickle.load(open(name_popular, "rb"))
data_for_predict = RecDataset.construct(interactions_df)
max_k = len(items_df["item_id"].unique())
sample_popular_user = data_for_predict.user_id_map.external_ids[0]
popular_list = list(
    loaded_popular.recommend(dataset=data_for_predict, users=[0], k=max_k, filter_viewed=False)["item_id"]
)

Теперь можно все сохранять в класс-обертку (она не сильно отличается от той, что была на лекции, но я добавил функцию для предсказания, чтобы использовать на проде) (`aerecommender.py`)

In [98]:
ae_recommender_model = AERecommender(X_pred, X_train, X_train, user_id2uid, popular_list)

In [99]:
ae_global_metrics = ae_recommender_model.evaluate()
ae_global_metrics

{'recall@5': 0.6427491784255025, 'recall@10': 0.75609232839576}

## Сохраним модель

In [100]:
name_ae = "../models/pickle_data/ae.pickle"
pickle.dump(ae_recommender_model, open(name_ae, "wb"))

# Для продакшна

In [101]:
name_ae = "../models/pickle_data/ae.pickle"
ae_loaded = pickle.load(open(name_ae, "rb"))

In [102]:
%%timeit
ae_loaded.recommend(21, 10)

94.1 ms ± 3.91 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Не знаю, прокатит ли, проверим